In [1]:
import keras as k
from keras.applications.vgg16 import VGG16
import pandas as pd
from keras.models import Model
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn import svm

Using TensorFlow backend.


In [2]:
# Extract and create the data set
data = pd.read_csv("data/micrograph.csv")

In [3]:
data_spheroidite = data.loc[data['primary_microconstituent'] == 'spheroidite'].copy()
data_network = data.loc[data['primary_microconstituent'] == 'network'].copy()
data_pearlite = data.loc[data['primary_microconstituent'] == 'pearlite'].copy()


img = image.img_to_array(image.load_img("data/micrograph/"+data_spheroidite['path'].iloc[0]))
img = img[0:484,:,:]
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)

In [4]:
model = VGG16(weights = 'imagenet', include_top=False)

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
layer_name = 'my_layer'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=[model.get_layer(index=3).output, 
                                          model.get_layer(index=6).output,
                                          model.get_layer(index=10).output,
                                          model.get_layer(index=14).output,
                                          model.get_layer(index=18).output])



In [6]:
def extract_features (data):
    data['c12'] = [np.empty(1) for i in range(data.shape[0])]
    data['c22'] = [np.empty(1) for i in range(data.shape[0])]
    data['c33'] = [np.empty(1) for i in range(data.shape[0])]
    data['c43'] = [np.empty(1) for i in range(data.shape[0])]
    data['c53'] = [np.empty(1) for i in range(data.shape[0])]
    
    for index, row in data.iterrows():
        img = image.img_to_array(image.load_img("data/micrograph/"+row['path']))
        img = img[0:484,:,:]
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)
        print(img.shape)
        c12, c22, c33, c43, c53 = intermediate_layer_model.predict(img)
        data.at[index,'c12'] = c12.mean(axis=(0,1,2,))
        data.at[index,'c22'] = c22.mean(axis=(0,1,2,))
        data.at[index,'c33'] = c33.mean(axis=(0,1,2,))
        data.at[index,'c43'] = c43.mean(axis=(0,1,2,))
        data.at[index,'c53'] = c53.mean(axis=(0,1,2,))
        break

In [7]:
%timeit
extract_features (data_spheroidite)
%timeit
extract_features (data_network)
%timeit
extract_features (data_pearlite)

(1, 484, 645, 3)
(1, 484, 645, 3)
(1, 484, 645, 3)


In [8]:
#train data

train_data_spher_netwk = pd.DataFrame ({'c12': np.append(data_spheroidite[0:100]['c12'].values, data_network[0:100]['c12'].values),
                                   'c22': np.append(data_spheroidite[0:100]['c22'].values, data_network[0:100]['c22'].values),
                                   'c33': np.append(data_spheroidite[0:100]['c33'].values, data_network[0:100]['c33'].values),
                                   'c43': np.append(data_spheroidite[0:100]['c43'].values, data_network[0:100]['c43'].values),
                                   'c53': np.append(data_spheroidite[0:100]['c53'].values, data_network[0:100]['c53'].values),
                                   'type': np.append(np.zeros(100), np.ones(100))}).sample(frac=1, random_state=0)

train_data_netwk_pearl = pd.DataFrame ({'c12': np.append(data_network[0:100]['c12'].values, data_pearlite[0:100]['c12'].values),
                                   'c22': np.append(data_network[0:100]['c22'].values, data_pearlite[0:100]['c22'].values),
                                   'c33': np.append(data_network[0:100]['c33'].values, data_pearlite[0:100]['c33'].values),
                                   'c43': np.append(data_network[0:100]['c43'].values, data_pearlite[0:100]['c43'].values),
                                   'c53': np.append(data_network[0:100]['c53'].values, data_pearlite[0:100]['c53'].values),
                                   'type': np.append(np.zeros(100), np.ones(100))}).sample(frac=1, random_state=0)

train_data_pearl_spher = pd.DataFrame ({'c12': np.append(data_pearlite[0:100]['c12'].values, data_spheroidite[0:100]['c12'].values),
                                   'c22': np.append(data_pearlite[0:100]['c22'].values, data_spheroidite[0:100]['c22'].values),
                                   'c33': np.append(data_pearlite[0:100]['c33'].values, data_spheroidite[0:100]['c33'].values),
                                   'c43': np.append(data_pearlite[0:100]['c43'].values, data_spheroidite[0:100]['c43'].values),
                                   'c53': np.append(data_pearlite[0:100]['c53'].values, data_spheroidite[0:100]['c53'].values),
                                   'type' : np.append(np.zeros(100), np.ones(100))}).sample(frac=1, random_state=0)


In [9]:
# test_data

test_data_spher_netwk = pd.DataFrame ({'c12': np.append(data_spheroidite[100:]['c12'].values, data_network[100:]['c12'].values),
                                   'c22': np.append(data_spheroidite[100:]['c22'].values, data_network[100:]['c22'].values),
                                   'c33': np.append(data_spheroidite[100:]['c33'].values, data_network[100:]['c33'].values),
                                   'c43': np.append(data_spheroidite[100:]['c43'].values, data_network[100:]['c43'].values),
                                   'c53': np.append(data_spheroidite[100:]['c53'].values, data_network[100:]['c53'].values),
                                   'type' : np.append(np.zeros(data_spheroidite[100:].shape[0]), np.ones(data_network[100:].shape[0]))})

test_data_netwk_pearl = pd.DataFrame ({'c12': np.append(data_network[100:]['c12'].values, data_pearlite[100:]['c12'].values),
                                   'c22': np.append(data_network[100:]['c22'].values, data_pearlite[100:]['c22'].values),
                                   'c33': np.append(data_network[100:]['c33'].values, data_pearlite[100:]['c33'].values),
                                   'c43': np.append(data_network[100:]['c43'].values, data_pearlite[100:]['c43'].values),
                                   'c53': np.append(data_network[100:]['c53'].values, data_pearlite[100:]['c53'].values),
                                   'type' : np.append(np.zeros(data_network[100:].shape[0]), np.ones(data_pearlite[100:].shape[0]))})

test_data_pearl_spher = pd.DataFrame ({'c12': np.append(data_pearlite[100:]['c12'].values, data_spheroidite[100:]['c12'].values),
                                   'c22': np.append(data_pearlite[100:]['c22'].values, data_spheroidite[100:]['c22'].values),
                                   'c33': np.append(data_pearlite[100:]['c33'].values, data_spheroidite[100:]['c33'].values),
                                   'c43': np.append(data_pearlite[100:]['c43'].values, data_spheroidite[100:]['c43'].values),
                                   'c53': np.append(data_pearlite[100:]['c53'].values, data_spheroidite[100:]['c53'].values),
                                   'type' : np.append(np.zeros(data_pearlite[100:].shape[0]), np.ones(data_spheroidite[100:].shape[0]))})


In [10]:
type(train_data_spher_netwk['type'].values)

numpy.ndarray

In [11]:
#spheroidite and network

# c12
sp_nt_clf12 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
scores = cross_validate(sp_nt_clf12, list(train_data_spher_netwk['c12'].values), list(train_data_spher_netwk['type'].values), cv=10)
#scores['test_recall_macro']
print ("c12 score : ", scores['test_score'].mean())

# c22
sp_nt_clf22 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
scores = cross_validate(sp_nt_clf22, list(train_data_spher_netwk['c22'].values), list(train_data_spher_netwk['type'].values), cv=10)
#scores['test_recall_macro']
print ("c22 score : ", scores['test_score'].mean())

# c33
sp_nt_clf33 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
scores = cross_validate(sp_nt_clf33, list(train_data_spher_netwk['c33'].values), list(train_data_spher_netwk['type'].values), cv=10)
#scores['test_recall_macro']
print ("c33 score : ", scores['test_score'].mean())

# c43
sp_nt_clf43 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
scores = cross_validate(sp_nt_clf43, list(train_data_spher_netwk['c43'].values), list(train_data_spher_netwk['type'].values), cv=10)
#scores['test_recall_macro']
print ("c43 score : ", scores['test_score'].mean())

# c53
sp_nt_clf53 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
scores = cross_validate(sp_nt_clf53, list(train_data_spher_netwk['c53'].values), list(train_data_spher_netwk['type'].values), cv=10)
#scores['test_recall_macro']
print ("c53 score : ", scores['test_score'].mean())

c12 score :  0.7050000000000001
c22 score :  0.905
c33 score :  0.975
c43 score :  0.985
c53 score :  0.99


In [12]:
#network and pearlite

# c12
nt_pl_clf12 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
scores = cross_validate(nt_pl_clf12, list(train_data_netwk_pearl['c12'].values), list(train_data_netwk_pearl['type'].values), cv=10)
#scores['test_recall_macro']
print ("c12 score : ", scores['test_score'].mean())

# c22
nt_pl_clf22 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
scores = cross_validate(nt_pl_clf22, list(train_data_netwk_pearl['c22'].values), list(train_data_netwk_pearl['type'].values), cv=10)
#scores['test_recall_macro']
print ("c22 score : ", scores['test_score'].mean())

# c33
nt_pl_clf33 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
scores = cross_validate(nt_pl_clf33, list(train_data_netwk_pearl['c33'].values), list(train_data_netwk_pearl['type'].values), cv=10)
#scores['test_recall_macro']
print ("c33 score : ", scores['test_score'].mean())

# c43
nt_pl_clf43 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
scores = cross_validate(nt_pl_clf43, list(train_data_netwk_pearl['c43'].values), list(train_data_netwk_pearl['type'].values), cv=10)
#scores['test_recall_macro']
print ("c43 score : ", scores['test_score'].mean())

# c53
nt_pl_clf53 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
scores = cross_validate(nt_pl_clf53, list(train_data_netwk_pearl['c53'].values), list(train_data_netwk_pearl['type'].values), cv=10)
#scores['test_recall_macro']
print ("c53 score : ", scores['test_score'].mean())

c12 score :  0.885
c22 score :  0.925
c33 score :  0.9650000000000001
c43 score :  0.99
c53 score :  1.0


In [13]:
#pearlite and spheroidite

# c12
pl_sp_clf12 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
scores = cross_validate(pl_sp_clf12, list(train_data_pearl_spher['c12'].values), list(train_data_pearl_spher['type'].values), cv=10)
#scores['test_recall_macro']
print ("c12 score : ", scores['test_score'].mean())

# c22
pl_sp_clf22 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
scores = cross_validate(pl_sp_clf22, list(train_data_pearl_spher['c22'].values), list(train_data_pearl_spher['type'].values), cv=10)
#scores['test_recall_macro']
print ("c22 score : ", scores['test_score'].mean())

# c33
pl_sp_clf33 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
scores = cross_validate(pl_sp_clf33, list(train_data_pearl_spher['c33'].values), list(train_data_pearl_spher['type'].values), cv=10)
#scores['test_recall_macro']
print ("c33 score : ", scores['test_score'].mean())

# c43
pl_sp_clf43 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
scores = cross_validate(pl_sp_clf43, list(train_data_pearl_spher['c43'].values), list(train_data_pearl_spher['type'].values), cv=10)
#scores['test_recall_macro']
print ("c43 score : ", scores['test_score'].mean())

# c53
pl_sp_clf53 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
scores = cross_validate(pl_sp_clf53, list(train_data_pearl_spher['c53'].values), list(train_data_pearl_spher['type'].values), cv=10)
#scores['test_recall_macro']
print ("c53 score : ", scores['test_score'].mean())

c12 score :  0.8350000000000002
c22 score :  0.885
c33 score :  0.9800000000000001
c43 score :  0.9899999999999999
c53 score :  0.99


In [14]:
sp_nt_clf53 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
sp_nt_clf53.fit(list(train_data_spher_netwk['c53'].values), list(train_data_spher_netwk['type'].values))
print(sp_nt_clf53.score(list(test_data_spher_netwk['c53'].values), list(test_data_spher_netwk['type'].values)))

0.9896373056994818


In [15]:
nt_pl_clf53 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
nt_pl_clf53.fit(list(train_data_netwk_pearl['c53'].values), list(train_data_netwk_pearl['type'].values))
print(nt_pl_clf53.score(list(test_data_netwk_pearl['c53'].values), list(test_data_netwk_pearl['type'].values)))

0.9926470588235294


In [16]:
pl_sp_clf53 = svm.SVC(kernel='rbf', gamma='scale', random_state=0)
pl_sp_clf53.fit(list(train_data_pearl_spher['c53'].values), list(train_data_pearl_spher['type'].values))
print(pl_sp_clf53.score(list(test_data_pearl_spher['c53'].values), list(test_data_pearl_spher['type'].values)))

0.9899328859060402


In [17]:
# For spherodite
pred_1 = sp_nt_clf53.predict(list(data_spheroidite[100:]['c53'].values))
pred_2 = pl_sp_clf53.predict(list(data_spheroidite[100:]['c53'].values))
correct_pred_sph = np.array(pred_1 == 0.0) & np.array(pred_2 == 1.0)

# For network
pred_1 = nt_pl_clf53.predict(list(data_network[100:]['c53'].values))
pred_2 = sp_nt_clf53.predict(list(data_network[100:]['c53'].values))
correct_pred_ntwk = np.array(pred_1 == 0.0) & np.array(pred_2 == 1.0)

# For pearlite
pred_1 = pl_sp_clf53.predict(list(data_pearlite[100:]['c53'].values))
pred_2 = nt_pl_clf53.predict(list(data_pearlite[100:]['c53'].values))
correct_pred_pl = np.array(pred_1 == 0.0) & np.array(pred_2 == 1.0)

In [18]:
prediction = np.append(np.append(correct_pred_sph, correct_pred_ntwk), correct_pred_pl)

total_error = prediction[prediction == True].shape[0]/prediction.shape[0]
print ("Total error : ", total_error)

Total error :  0.9829268292682927


In [115]:
data_spheroidite.to_csv('data/sperodite.csv', sep=',')
data_network.to_csv('data/network.csv', sep=',')
data_pearlite.to_csv('data/pearlite.csv', sep=',')

In [11]:
data_spheroidite.to_csv('data/sperodite_t.csv', sep='\t')
data_network.to_csv('data/network_t.csv', sep='\t')
data_pearlite.to_csv('data/pearlite_t.csv', sep='\t')

In [33]:
data_spheroidite['path'].iloc[0]

'micrograph2.tif'

In [37]:
?cross_validate

In [20]:
intermediate_output.shape

(1, 242, 322, 64)

In [52]:
a = [5, 6, 7, 8]
df = pd.DataFrame({"a": [5, 6 , 7 , 8], "b": [5, 6, 7, 8]})

In [62]:
df.sample(frac=1, random_state=0)

,a,b
2,7,7
3,8,8
1,6,6
0,5,5


In [39]:
df1 = pd.DataFrame({"a": np.append(y,y)})

In [40]:
df1

,a
0,5
1,6
2,5
3,6


In [117]:
def sam(df):
    for index, row in df.iterrows():
        print(index)
        row['a']=5

In [118]:
sam(df)

0
1


In [153]:
[x for x in range(0,100)]

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99]

In [44]:
list(train_data_spher_netwk['c12'].values)[1]

array([341.3344  , 185.11885 , 150.824   , 112.44451 , 327.8188  ,
       275.09537 , 198.18904 , 102.41569 , 242.96877 , 136.56586 ,
       371.4226  , 115.533806, 225.17302 , 275.53955 ,  62.7535  ,
       116.48405 , 227.53041 , 272.0164  , 102.508705, 218.12967 ,
       336.26205 , 146.76839 , 143.13101 ,  22.458551, 185.24213 ,
       112.42335 , 245.81418 , 354.76944 , 238.9223  , 318.40817 ,
       276.80017 ,  75.56907 , 203.20589 , 224.17995 , 201.61795 ,
       207.20468 , 162.43324 , 297.76343 , 149.47534 ,  98.867065,
       274.83307 , 213.31392 , 256.21606 , 232.32637 , 216.25096 ,
        43.664513, 187.11922 , 392.18225 , 142.88338 ,  10.734444,
       340.461   , 115.634865, 317.44736 , 323.36084 ,   3.922971,
       326.60004 , 279.1217  , 253.99638 , 255.02556 , 299.32828 ,
        71.582535,  82.9605  , 293.3207  , 207.816   ], dtype=float32)

array([306.60367 , 173.878   , 144.62267 , 103.02746 , 373.48346 ,
       234.50632 , 186.48154 ,  95.426   , 224.66208 , 143.64041 ,
       332.59692 ,  80.73082 , 227.8951  , 248.49567 ,  60.715126,
       101.90525 , 213.87029 , 290.9517  ,  90.678696, 127.320366,
       292.6236  , 143.44925 , 142.985   ,  24.63697 , 174.84714 ,
       154.93318 , 265.9502  , 383.0049  , 237.12733 , 242.7477  ,
       195.99959 ,  99.466385, 191.7526  , 207.61232 , 244.82773 ,
       135.08246 , 139.2994  , 316.17105 , 133.88344 ,  86.484886,
       280.5803  , 213.57915 , 226.80736 , 207.57018 , 208.27716 ,
        43.500416, 196.26704 , 367.94745 , 126.26086 ,  12.095521,
       285.14584 , 102.63821 , 316.3713  , 325.63885 ,  15.359129,
       243.88461 , 271.18744 , 180.01408 , 250.63356 , 214.83092 ,
        65.89104 ,  75.54554 , 213.31656 , 211.80426 ], dtype=float32)

In [113]:
a = np.array([1, 0, 0 , 1])
b = np.array([0, 1, 0 , 0])
np.array(a == 1) & np.array(b == 0)

array([ True, False, False,  True])

In [103]:
k = a == 1

In [93]:
l = b == 0

In [94]:
k

array([ True, False, False,  True])

In [95]:
l

array([ True, False,  True,  True])

In [105]:
k & l

array([ True, False, False,  True])